In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
import json

In [2]:
df1 = pd.read_excel('BLUSOL/CLIENTES BLUSOL 2019.xlsx')
df2 = pd.read_excel('BLUSOL/CLIENTES BLUSOL 2020.xlsx')
df3 = pd.read_excel('BLUSOL/CLIENTES BLUSOL 2021 A 2023.xlsx')
df4 = pd.read_excel('BancoEmpreendedor/CLIENTES BE 2019 A 2023.xlsx')
df5 = pd.read_excel('crecerto/CLIENTES CRECERTO 2019 A 2023.xlsx')
print(len(df1),len(df2),len(df3),len(df4),len(df5))

11196 12823 24435 32437 31555


In [3]:
df = pd.concat([df1,df2,df3,df4,df5]).reset_index(drop = True)

In [4]:
len(df)

112446

In [5]:
for i in range(3):
    cols = list(df.columns)
    res = [(a, b) for idx, a in enumerate(cols) for b in cols[idx + 1:]]
    deleted = set()
    for x,y in res:
        if (x in deleted) or (y in deleted):
            continue
        newx = x.replace('Nº','nro').replace('PF/PJ','pessoaFisicaJuridica').replace('Score','conceito').replace('Data conceito SPC','datadospc').replace('numero','nro').replace('Classificacao Empresa','classificacao')
        newy = y.replace('Nº','nro').replace('PF/PJ','pessoaFisicaJuridica').replace('Score','conceito').replace('Data conceito SPC','datadospc').replace('numero','nro').replace('Classificacao Empresa','classificacao')
        if newx.lower().replace(' ','') == newy.lower().replace(' ',''):
            new_col = df[x].fillna(df[y])
            del(df[x])
            deleted.add(x)
            del(df[y])
            deleted.add(y)
            df[newx.lower().replace(' ','')] = new_col

In [6]:
for column in df.columns:
    if df[column].dtype == 'object':
        uniq = df[column].unique()
        if len(uniq) > 10:
            print(column,':',len(uniq))
        else:
            print(column,':',list(uniq))

CEP : 19521
situacaodoimovelderesidencia : ['Propria', 'Alugada', 'Cedida', 'Outros']
situacaoimoveldeatividade : ['Propria', 'Alugada', 'Outros']
tipodeponto : ['Ambulante', 'Fixo']
formacaoescolar : ['Fundamental', 'Médio', 'Superior', 'Pós-Graduação', 'Sem Instrução', 'Outros']
atividadeexercida : 3324
constituicao : ['Informal', 'M.E.I.', 'Microempresa', 'Outros', 'E.P.P.']
pessoafisicajuridica : ['F', 'J', nan]
classificacao : ['AA', 'A', 'D', 'H', 'C', 'B', 'E', 'G', 'F', nan]
produto : ['MICROCRÉDITO', 'MICROCRÉDITO SOCIAL', 'BLUSOL CONSTRUIR', 'BLUSOL REFORMAR', 'CRÉDITO SOCIAL', 'CONSUMO', 'CHEQUES']
finalidadeemprestimo : ['Giro', 'Fixo', 'Misto', 'Outros']
dataemprestimo : 2201
dataliberacao : 2191
datacadastro : 98310
primeirovencimento : 3013
situacaocredito : ['Aprovado', 'Estornado']
bairro : 3440
cidade : 614
cepempresa : 18167
bairroempresa : 3091
cidadeempresa : 644
conceitospc : ['S', nan, 'N', 'B', 'D', 'F', 'C', 'A', 'E']
datadospc : 2026


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112446 entries, 0 to 112445
Data columns (total 38 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   finalDoContrato               3123 non-null    float64
 1   CEP                           112377 non-null  object 
 2   idcliente                     112446 non-null  float64
 3   tempoderesidenciaanos         24241 non-null   float64
 4   tempoderesidenciameses        3233 non-null    float64
 5   situacaodoimovelderesidencia  112446 non-null  object 
 6   nropessoasnacasa              72705 non-null   float64
 7   situacaoimoveldeatividade     112446 non-null  object 
 8   temporesidenciaimovel         1031 non-null    float64
 9   tipodeponto                   112446 non-null  object 
 10  formacaoescolar               112446 non-null  object 
 11  atividadeexercida             112359 non-null  object 
 12  tempodeatividade              23026 non-null

In [8]:
df.columns

Index(['finalDoContrato', 'CEP', 'idcliente', 'tempoderesidenciaanos',
       'tempoderesidenciameses', 'situacaodoimovelderesidencia',
       'nropessoasnacasa', 'situacaoimoveldeatividade',
       'temporesidenciaimovel', 'tipodeponto', 'formacaoescolar',
       'atividadeexercida', 'tempodeatividade', 'constituicao',
       'pessoafisicajuridica', 'nrodefuncionarios', 'rendafamiliarmensal',
       'totaldasreceitas', 'classificacao', 'produto', 'finalidadeemprestimo',
       'taxaaomes', 'valoremprestado', 'quantidadeparcelas',
       'nroparcelaspagas', 'dataemprestimo', 'dataliberacao', 'datacadastro',
       'primeirovencimento', 'maioratrasopagas', 'situacaocredito', 'bairro',
       'cidade', 'cepempresa', 'bairroempresa', 'cidadeempresa', 'conceitospc',
       'datadospc'],
      dtype='object')

In [10]:
def corrigir_taxa(x):
    if x >= 1000000:
        return x / 100000000
    return x / 100

def valor_da_parcela(row):
    taxa = row['taxaaomes']
    vp = row['valoremprestado']
    n = row['quantidadeparcelas']
    if taxa == 0:
        return vp / n
    return vp * (taxa * ((1 + taxa) ** n)) / (((1 + taxa) ** n) - 1)

def renda(row):
    if pd.isnull(row['totaldasreceitas']):
        if pd.isnull(row['rendafamiliarmensal']):
            return 0
        return row['rendafamiliarmensal']
    if pd.isnull(row['rendafamiliarmensal']):
        return row['totaldasreceitas']
    return max(row['totaldasreceitas'],row['rendafamiliarmensal'])

In [11]:
backup = df.copy()

In [12]:
df = backup.copy()

In [13]:
df = df[df['situacaocredito'] != 'Estornado']
df = df[df['quantidadeparcelas'] != 0]
df = df[~df['primeirovencimento'].isna()]
df = df[~df['nroparcelaspagas'].isna()]
df['primeirovencimento'] = pd.to_datetime(df['primeirovencimento'])
df['ultimaparcelapaga'] = df['primeirovencimento'] + df['nroparcelaspagas'].apply(lambda x: relativedelta(months = x - 1))
today = datetime(2023,4,20)
df['diasdesdeultimaparcela'] = (today - df['ultimaparcelapaga']).apply(lambda x: x.days)
df['taxaaomes'] = df['taxaaomes'].apply(corrigir_taxa)
df['valorparcela'] = df.apply(valor_da_parcela,axis = 1)
df['renda'] = df.apply(renda,axis = 1)
df['pesoparcela'] = df['valorparcela'] / df['renda']
df['pesoparcela'] = df['pesoparcela'].apply(lambda x: 1 if x > 1 else x)
df['DEFAULT'] = (df['diasdesdeultimaparcela'] > 90) & (df['nroparcelaspagas'] != df['quantidadeparcelas'])
df = df.drop(['atividadeexercida','idcliente','nroparcelaspagas','primeirovencimento',
              'situacaocredito','ultimaparcelapaga','diasdesdeultimaparcela','renda',
              'dataemprestimo','dataliberacao','datacadastro','primeirovencimento','maioratrasopagas','CEP','bairro','cidade',
              'cepempresa','bairroempresa','cidadeempresa','datadospc','finalDoContrato','classificacao','produto'],axis = 1)

C:\Users\aruav\AppData\Local\Temp\ipykernel_9408\655652845.py:6: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  df['ultimaparcelapaga'] = df['primeirovencimento'] + df['nroparcelaspagas'].apply(lambda x: relativedelta(months = x - 1))


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104482 entries, 0 to 112445
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   tempoderesidenciaanos         22120 non-null   float64
 1   tempoderesidenciameses        3026 non-null    float64
 2   situacaodoimovelderesidencia  104482 non-null  object 
 3   nropessoasnacasa              67216 non-null   float64
 4   situacaoimoveldeatividade     104482 non-null  object 
 5   temporesidenciaimovel         893 non-null     float64
 6   tipodeponto                   104482 non-null  object 
 7   formacaoescolar               104482 non-null  object 
 8   tempodeatividade              21529 non-null   float64
 9   constituicao                  104482 non-null  object 
 10  pessoafisicajuridica          32957 non-null   object 
 11  nrodefuncionarios             70985 non-null   float64
 12  rendafamiliarmensal           104323 non-null  fl

In [15]:
nafills = {}
mandatory = []
limit = 80000
for col in df.drop(['pesoparcela','valorparcela'],axis = 1).columns:
    nacount = df[col].isna().sum()
    if df[col].dtype in ['float64','int64']:
        if nacount > limit:
            del(df[col])
        elif nacount > 0:
            median = df[col].median()
            df[col] = df[col].fillna(median)
            nafills[col] = median
        else:
            mandatory.append(col)
    elif df[col].dtype == 'object':
        if df[col].isna().sum() > limit:
            del(df[col])
        elif nacount > 0:
            df[col] = df[col].fillna('Não Informado')
            df[col] = df[col].apply(str)
            nafills[col] = 'Não Informado'
        else:
            mandatory.append(col)

with open('nafill.json','w',encoding="utf-8") as file:
    json.dump(nafills,file,ensure_ascii = False)

with open('mandatory.json','w',encoding="utf-8") as file:
    json.dump(mandatory,file,ensure_ascii = False)

In [16]:
order = list(df.drop(['DEFAULT'],axis = 1).columns)
with open('ordem_atributos.json','w',encoding="utf-8") as file:
    json.dump(order,file,ensure_ascii = False)

In [263]:
# df = df.drop(['valoremprestado','quantidadeparcelas'],axis = 1)

In [264]:
df = df.reset_index(drop = True)

In [265]:
permited_values = {}
for col in df.columns:
    if df[col].dtype == 'object':
        permited = list(df[col].unique())
        try:
            permited.remove('Não Informado')
        except ValueError:
            pass
        permited_values[col] = permited

with open('permited_values.json','w',encoding="utf-8") as file:
    json.dump(permited_values,file,ensure_ascii = False)

In [266]:
catdict = {}
for column in df.columns:
    if df[column].dtype == 'object':
        cat = {}
        uniq = df[column].unique()
        for value in uniq:
            sample = df.loc[df[column] == value].sample(frac = 0.5)
            cat[value] = sample['DEFAULT'].sum() / len(sample)
        catdict[column] = cat
with open('categories.json','w',encoding="utf-8") as file:
    json.dump(catdict,file,ensure_ascii = False)

In [267]:
catdict = json.load(open('categories.json','r',encoding = 'utf-8'))
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].apply(lambda x: catdict[column][x])

In [268]:
normalize_dict = {}
for column in df.drop('DEFAULT',axis = 1).columns:
    normalize_dict[column] = {'mean':df[column].mean(),'std':df[column].std()}
with open('normalization.json','w',encoding="utf-8") as file:
    json.dump(normalize_dict,file,ensure_ascii = False)

In [269]:
normalize_dict = json.load(open('normalization.json','r',encoding = 'utf-8'))
for column in df.drop('DEFAULT',axis = 1).columns:
    df[column] = (df[column] - normalize_dict[column]['mean']) / normalize_dict[column]['std']

In [270]:
df.to_feather('big_frame_4.feather')